In [347]:
import numpy as np 
import pandas as pd 
from sklearn.linear_model import Ridge as R
from sklearn.model_selection import KFold as KF, cross_val_score as cvs

/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv
/kaggle/input/house-prices-advanced-regression-techniques/train.csv


In [348]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
test["SalePrice"] = np.zeros(len(test.index))
ids = test["Id"]
train = train.drop(["Id"], axis = 1)
test = test.drop(["Id"], axis = 1)
train = train[train.dtypes[train.dtypes.isin(["Int64", "Float64"])].index]
test = test[train.columns]
for column in train.drop("SalePrice", axis = 1).columns:
    train[column] = [(x-train[column].min())/(train[column].max()-train[column].min()) for x in train[column]]
    test[column] = [(x-test[column].min())/(test[column].max()-test[column].min()) for x in test[column]]
train = train[train.corr()["SalePrice"][abs(train.corr()["SalePrice"]) > 0.25].index]
df = train.append(test[train.columns])
df = df.drop(df.isnull().sum()[df.isnull().sum() > 2].index, axis = 1)
df[df.isnull().sum()[df.isnull().sum() != 0].index] = df[df.isnull().sum()[df.isnull().sum() != 0].index].fillna(np.mean(df[df.isnull().sum()[df.isnull().sum() != 0].index]))
train = df[df["SalePrice"] != 0]
test = df[df["SalePrice"] == 0]

In [349]:
X_train = train.drop("SalePrice", axis = 1)
X_test = test.drop("SalePrice", axis = 1)
y_train = train["SalePrice"]
kf = KF(shuffle = True, n_splits = 5)
def alpha(X_train, y_train):
    scores = []
    for i in [10**x for x in np.linspace(-2, 2, 5)]:
        model = R(alpha = i)
        scores.append(np.mean(cvs(model, X_train, y_train, cv = kf)))
    model = R(alpha = sorted(scores)[-1]).fit(X_train, y_train)
    return {i: np.mean(cvs(model, X_train, y_train, cv = kf))}
alphavalue = alpha(X_train, y_train)
model = R(alpha = list(alphavalue.keys())[0]).fit(X_train, y_train)
y_test = model.predict(X_test)
pred = pd.DataFrame({"Id": ids, "SalePrice": y_test}).set_index("Id")
pred.to_csv("submission.csv")